<a href="https://colab.research.google.com/github/PrayaniSingh0106/NetflixMovieRatingPredictionSystem/blob/master/year_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install omdb
import pandas as pd
import json
from omdb import OMDBClient
import math
import os
from scipy import stats
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import sys
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

In [0]:
#make the movie_year_netflix in decade form

f = pd.read_csv('/content/gdrive/My Drive/ALDA/Project/merged.csv',header=None,names = ['Movie_ID', 'Netflix_Year','Title'],encoding='latin-1',usecols=[12,13,22])
client = OMDBClient(apikey="21dc1eaa")
df.head()
#df['Netflix_Year'] = df['Netflix_Year'].astype(float)
df = df.sort_values(by=['Netflix_Year'])
df['Netflix_Year'] = df['Netflix_Year'].str.extract('(\d\d\d)\d',expand=True) + '0';
df.head()
#df['Netflix_Year'].min()
#write_path = '/content/gdrive/My Drive/ALDA/Project/year_filtered.csv'

,Movie_ID,Netflix_Year,Title
5514,7654,1890,The LumiÃÂÃÂ¨re Brothers' First Films
6553,9103,1910,Tillie's Punctured Romance
7804,10898,1910,Cabiria
2264,3137,1910,Les vampires
6349,8821,1910,The Birth of a Nation


In [0]:
df = pd.read_csv('/content/gdrive/My Drive/ALDA/Project/classifier_final_df.csv',header=0,encoding='latin-1')
df['movie_year_netflix'] = df['movie_year_netflix'].astype(str).str.extract('(\d\d\d)\d',expand=True)+'0';
df.to_csv('/content/gdrive/My Drive/ALDA/Project/classifier_final_file.csv')
df.head()
df = df.iloc[:,1:]
df.head()

,imdb_rating,imdb_votes,movie_id,movie_year_netflix,runtime,Documentary,Crime,Musical,Sci-Fi,Comedy,Talk-Show,History,Game-Show,Music,Short,Reality-TV,News,Adult,Family,Biography,Western,War,Horror,Sport,Adventure,Romance,Action,Thriller,Drama,Film-Noir,Fantasy,Animation,Mystery,Rating,APPROVED,Approved,E,G,GP,M,M/PG,NC-17,NOT RATED,NR,Not Rated,PASSED,PG,PG-13,Passed,R,TV-13,TV-14,TV-G,TV-MA,TV-PG,TV-Y,TV-Y7,TV-Y7-FV,UNRATED,Unrated,X
0,7.7,410.0,1,2000,50.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3.749543,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,7.7,10015.0,3,1990,122.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,3.641153,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,6.2,39.0,6,1990,15.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3.084396,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,5.0,62.0,7,1990,83.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2.129032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,6.5,1394.0,10,2000,100.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,3.180723,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#onehot_encoder for "rated" field
feature = df['rated'].values.reshape(df.shape[0], 1)
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
feature = onehot_encoder.fit_transform(feature).astype('int')
lf=list(onehot_encoder.categories_[0])
#print(feature)
#print(np.unique(feature))
tempdf=pd.DataFrame(data=feature,columns=lf)
df4 = pd.concat([df,tempdf],axis=1)
#df4.to_csv('/content/gdrive/My Drive/ALDA/Project/classifier_final_file1.csv')
df4 = df4.drop(['rated'],axis=1)
df4 = df4.iloc[:,1:]
df4.head()
df4.to_csv('/content/gdrive/My Drive/ALDA/Project/classifier_final_df.csv',index = None)

In [0]:
df4.head()

,Unnamed: 0,imdb_rating,imdb_votes,movie_id,movie_year_netflix,rated,runtime,Documentary,Crime,Musical,Sci-Fi,Comedy,Talk-Show,History,Game-Show,Music,Short,Reality-TV,News,Adult,Family,Biography,Western,War,Horror,Sport,Adventure,Romance,Action,Thriller,Drama,Film-Noir,Fantasy,Animation,Mystery,Rating,APPROVED,Approved,E,G,GP,M,M/PG,NC-17,NOT RATED,NR,Not Rated,PASSED,PG,PG-13,Passed,R,TV-13,TV-14,TV-G,TV-MA,TV-PG,TV-Y,TV-Y7,TV-Y7-FV,UNRATED,Unrated,X
0,0,7.7,410.0,1,2000,R,50.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3.749543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,7.7,10015.0,3,1990,R,122.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,3.641153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,6.2,39.0,6,1990,R,15.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3.084396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,5.0,62.0,7,1990,R,83.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2.129032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,6.5,1394.0,10,2000,R,100.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,3.180723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
#Concat files to find correlation between awards, rated, movie_year_netflix, genre and avrage rating

df1 = pd.read_csv('/content/gdrive/My Drive/ALDA/Project/year_filter.csv',header=0,encoding='latin-1')
df2 = pd.read_csv('/content/gdrive/My Drive/ALDA/Project/merged.csv',header=0,encoding='latin-1')
df3 = pd.concat([df1['movie_year_netflix'], df1['rated'], df1['Primetime Emmy Won'], df1['Oscar Won'],
                 df1['Oscar Nomination'], df1['Golden Globe Won'], df1['Golden Globe Nomination'], df1['BAFTA Film Award Won'], df1['BAFTA Film Award Nomination'],
                 df1['Other wins'], df1['Other nominations'], df2['genre'], df1['Rating']], 
                axis=1, keys=['movie_year_netflix', 'rated', 'Primetime Emmy Won','Oscar Won',
                              'Oscar Nomination', 'Golden Globe Won', 'Golden Globe Nomination', 'BAFTA Film Award Won',
                              'BAFTA Film Award Nomination', 'Other wins', 'Other nominations', 'genre', 'Rating'])
df3.to_csv('/content/gdrive/My Drive/ALDA/Project/feature_selection.csv')

In [0]:
#ANOVA & Pearson Correlation

col_list=[df3['movie_year_netflix'].dropna(),df3['rated'].dropna(),df3['genre'].dropna()]
columns=['movie_year_netflix','rated','genre']
ls=[]
for l,i in zip(col_list,columns):
    ls=[]
    for x in l:
        ls.append(df3['Rating'][df3[i]==x])
    #print(ls)
    print('F value for',i,'is',stats.f_oneway(*ls)[0])

x = scipy.stats.pearsonr(df3['Primetime Emmy Won'].dropna(), df3['Rating'].dropna().astype(int))
print('Correlation between Primetime Emmy Won and rating is: ', x)

x = scipy.stats.pearsonr(df3['Oscar Won'].dropna(), df3['Rating'].dropna().astype(int))
print('Correlation between Oscar Won and rating is: ', x)

x = scipy.stats.pearsonr(df3['Oscar Nomination'].dropna(), df3['Rating'].dropna().astype(int))
print('Correlation between Oscar Nomination and rating is: ', x)

x = scipy.stats.pearsonr(df3['Golden Globe Won'].dropna(), df3['Rating'].dropna().astype(int))
print('Correlation between Golden Globe Won and rating is: ', x)

x = scipy.stats.pearsonr(df3['Golden Globe Nomination'].dropna(), df3['Rating'].dropna().astype(int))
print('Correlation between Golden Globe Nomination and rating is: ', x)

x = scipy.stats.pearsonr(df3['BAFTA Film Award Won'].dropna(), df3['Rating'].dropna().astype(int))
print('Correlation between BAFTA Film Award Won and rating is: ', x)

x = scipy.stats.pearsonr(df3['BAFTA Film Award Nomination'].dropna(), df3['Rating'].dropna().astype(int))
print('Correlation between BAFTA Film Award Nomination and rating is: ', x)

x = scipy.stats.pearsonr(df3['Other wins'].dropna(), df3['Rating'].dropna().astype(int))
print('Correlation between Other wins and rating is: ', x)

x = scipy.stats.pearsonr(df3['Other nominations'].dropna(), df3['Rating'].dropna().astype(int))
print('Correlation between Other nominations and rating is: ', x)


F value for movie_year_netflix is 35.611641891164254
F value for rated is 89.50147217256163
F value for genre is nan
Correlation between Primetime Emmy Won and rating is:  (0.04048573217672645, 5.573525254112072e-06)
Correlation between Oscar Won and rating is:  (0.14494930406361722, 5.037123247043525e-60)
Correlation between Oscar Nomination and rating is:  (0.14814713556649653, 1.194111733445347e-62)
Correlation between Golden Globe Won and rating is:  (0.03367590128371205, 0.00015840732806566814)
Correlation between Golden Globe Nomination and rating is:  (0.06054047815676673, 1.0810622121865493e-11)
Correlation between BAFTA Film Award Won and rating is:  (0.012717885919105984, 0.15379670237130932)
Correlation between BAFTA Film Award Nomination and rating is:  (0.039581942942959476, 8.975828094531536e-06)
Correlation between Other wins and rating is:  (0.1756736351048378, 1.0031176725608734e-87)
Correlation between Other nominations and rating is:  (0.16411509087668014, 1.18907275

In [0]:
#Normalize

df = pd.read_csv('/content/gdrive/My Drive/ALDA/Project/final_dataset_unnormalized.csv',header=0,encoding='latin-1')

x_array = np.array(df['imdb_votes'])
x = preprocessing.normalize([x_array])
df['imdb_votes'] = x[0]

x_array = np.array(df['imdb_rating'])
x = preprocessing.normalize([x_array])
df['imdb_rating'] = x[0]

x_array = np.array(df['movie_year_netflix'])
x = preprocessing.normalize([x_array])
df['movie_year_netflix'] = x[0]

x_array = np.array(df['runtime'])
x = preprocessing.normalize([x_array])
df['runtime'] = x[0]

# x_array = np.array(df['Rating'])
# x = preprocessing.normalize([x_array])
# df['Rating'] = x[0]

df.head()
df.to_csv('/content/gdrive/My Drive/ALDA/Project/final_dataset.csv', index = None)


In [0]:
def preprocess(path):
  df=pd.read_csv(path,header=None,names = ['Cust_Id', 'Rating','Date'])
  ls=[]
  df1=df[df['Rating'].isnull()==True]
  cols=list(df1.index)
  m_id=list(df1['Cust_Id'].values)
  for i in range(len(cols)-1):
    c=m_id[i].strip(':')
    c=int(c)
    temp=[c]*(cols[i+1]-cols[i])
    ls.extend(temp)
  c=m_id[len(cols)-1].strip(':')
  c=int(c)
  temp=[c]*(df.shape[0]-cols[len(cols)-1])
  ls.extend(temp)
  df['movie_id']=ls
  df_final=df.drop(cols,axis=0)
  return df_final

In [0]:
df_1=preprocess('/content/gdrive/My Drive/ALDA/Project/combined_data_1.txt.zip')

In [0]:
df_2=preprocess('/content/gdrive/My Drive/ALDA/Project/combined_data_2.txt.zip')

In [0]:
df_3=preprocess('/content/gdrive/My Drive/ALDA/Project/combined_data_3.txt.zip')

In [0]:
df_4=preprocess('/content/gdrive/My Drive/ALDA/Project/combined_data_4.txt.zip')

In [0]:
print(df_1.shape,df_2.shape,df_3.shape,df_4.shape)
df_1.head()

(24053764, 4) (26977591, 4) (22601629, 4) (26847523, 4)


,Cust_Id,Rating,Date,movie_id
1,1488844,3.0,2005-09-06,1
2,822109,5.0,2005-05-13,1
3,885013,4.0,2005-10-19,1
4,30878,4.0,2005-12-26,1
5,823519,3.0,2004-05-03,1


In [0]:
df = pd.concat([df_1,df_2,df_3,df_4],axis=0)
df.head()
print(df.shape)





(100480507, 4)


In [0]:
df.to_csv('/content/gdrive/My Drive/ALDA/Project/combined_data_all.csv', index = None)

In [0]:
df_all_cust = pd.read_csv('/content/gdrive/My Drive/ALDA/Project/combined_data_all.csv',header=0,encoding='latin-1')

In [0]:
#from here
#find list of movies for a user

def user_movies(cust_id,movie_id):
  if cust_id in df_all_cust.Cust_Id.values:
    x = df_all_cust[df_all_cust['Cust_Id']==cust_id]
    print (x)
  return x

In [0]:

df_cluster = pd.read_csv('/content/gdrive/My Drive/ALDA/Project/cluster_movie.csv',header=None,names = ['Movie_ID', 'Cluster_ID'],encoding='latin-1')

In [0]:
df_rating = pd.read_csv('/content/gdrive/My Drive/ALDA/Project/avg_rat_movie.csv',header = 0, names = ['Movie_ID', 'Rating'],encoding='latin-1')
print(df_rating.head())


,movie_id,Rating
0,1,3.749543
1,3,3.641153
2,6,3.084396
3,7,2.129032
4,10,3.180723


In [0]:
df_user_movies = user_movies(30878,3)
df_all_cust['Cust_Id'].unique().shape
df_all_cust.head()
movie_id = 3
df_x = df_cluster.loc[df_cluster['Movie_ID'] == movie_id]
cluster_id = df_x['Cluster_ID'].values[0]
#print('Cluster Id : ',cluster_id)
mov_list=[]

for index, row in df_user_movies.iterrows():
    #print(row['movie_id'])
    df_clus = df_cluster.loc[df_cluster['Movie_ID'] == row['movie_id']]
    cluster_id_cur = df_clus['Cluster_ID']
    #print(cluster_id_cur.shape)
    #print(cluster_id_cur.values)
    if len(cluster_id_cur.values) != 0:
      if cluster_id_cur.values[0]==cluster_id:
        mov_list.append(row['movie_id'])
df_user_movies = df_user_movies[df_user_movies['movie_id'].isin(mov_list)]


#avg rating of input movie 
average_rating_ip_mov = df_rating.loc[df_rating['Movie_ID'] == 3]
print('average_rating_ip_mov: ', average_rating_ip_mov['Rating'])

for index, row in df_user_movies.iterrows():
    df_movie_rat = df_rating[df_rating['Movie_ID'] == row['movie_id']]
    #print(df_movie_rat.values)
    df_rating_mov = df_movie_rat['Rating']
    #print(df_rating_mov.values)
    #index = df_user_movies.index[df_user_movies['movie_id'] == row['movie_id']].tolist()
    df_user_movies.loc[index, 'Average Rating'] = df_rating_mov.values[0]
df_user_movies.head()
diff = 0
count = 1
mean = 1;

for index, row in df_user_movies.iterrows():
  user_rat = row['Rating']
  avg_rat = row['Average Rating']
  diff = abs(avg_rat - user_rat) + diff
  count = count + 1

mean = diff/count
print(mean)

val = average_rating_ip_mov['Rating'].values[0] + (mean * correlation_user[0])
print("Metric ", val)

           Cust_Id  Rating        Date  movie_id
3            30878     4.0  2005-12-26         1
2849         30878     1.0  2005-04-27         5
31469        30878     3.0  2004-10-19        18
52546        30878     3.0  2004-08-08        28
92838        30878     4.0  2004-06-10        30
...            ...     ...         ...       ...
100248936    30878     4.0  2004-02-02     17709
100326927    30878     2.0  2004-01-25     17730
100347192    30878     4.0  2004-03-02     17746
100371348    30878     4.0  2004-02-16     17762
100405451    30878     4.0  2004-02-16     17764

[1291 rows x 4 columns]
   Movie_ID    Rating
0         1  3.749543
1         3  3.641153
2         6  3.084396
3         7  2.129032
4        10  3.180723
average_rating_ip_mov:  1    3.641153
Name: Rating, dtype: float64
0.5920075091086957
Metric  3.821166288677005


In [0]:
#finding correlation
correlation_user = scipy.stats.pearsonr(df_user_movies['Rating'], df_user_movies['Average Rating'].astype(int))
print('Correlation between User Rating and Average Rating is: ', x)

In [0]:
df_user_movies.head()

,Cust_Id,Rating,Date,movie_id
3,30878,4.0,2005-12-26,1
2849,30878,1.0,2005-04-27,5
31469,30878,3.0,2004-10-19,18
52546,30878,3.0,2004-08-08,28
92838,30878,4.0,2004-06-10,30


In [0]:
df_cluster.head()

,0,1
0,1,3
1,3,2
2,6,2
3,7,3
4,10,2
